In [1]:
from langchain_community.document_loaders import Docx2txtLoader   
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings 
from langchain_chroma import Chroma

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   # 토큰수
    chunk_overlap=200, # 겹치는 토큰수
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# 임베딩 결과를 /chroma 디렉토리에 저장
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma-tax", persist_directory="./chroma")

query = '한새 비목의 설치에 대한 동의 요청이 소관 상임위원회에 회부되어 회부된 때부터 몇시간 이내에 동의 여부 를 결정하여야 합니까?'
retrieved_docs = database.similarity_search(query, k=3)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")
prompt = f"""[Identity]
- 당신은 최고의 법률 전문가입니다.
- [Context]를 참고하여 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""
ai_message = llm.invoke(prompt)

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-4o")
question = "인프런에는 어떤 강의가 있나요?"
text_prompt = f"아래의 질문에 답변해주세요:\n\n {question}"

prompt_with_template = "아래의 질문에 답변해주세요:\n\n {question}"
prompt_template = PromptTemplate(template=prompt_with_template, 
                                 input_variables=["question"] )
prompt_chain = prompt_template | llm
prompt_chain.invoke({"question": question})

loader = WebBaseLoader("https://www.inflearn.com/course/rag-llm-application개발-langchain/dashboard")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   # 토큰수
    chunk_overlap=200, # 겹치는 토큰수
)

document_list = loader.load_and_split(text_splitter=text_splitter)
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
    collection_name="inflearn-lectures",
    persist_directory="./inflearn-lectures",
)

retriever = database.as_retriever(search_kwargs={"k": 4})
retriever.invoke(question)
retrived_docs = retriever.invoke(question)

rag_prompt_template = PromptTemplate(
    template='''
    you are a helpful assistant that can answer questions about the Inflearn website.
    You are given the following context:
    {context}
    Question: {question}
    ''',
    input_variables=["context", "question"]
)

rag_chain = rag_prompt_template | llm
rag_chain.invoke({"context": retrived_docs, "question": question})


AIMessage(content='인프런에는 "RAG를 활용한 LLM Application 개발 (feat. LangChain)"이라는 강의가 있습니다. 이 강의는 실리콘밸리 GenAI 해커톤 우승자인 강병진 강사가 진행하며, RAG(개방형 정보 저장 및 검색)와 관련된 다양한 데이터 전처리 기술 및 검색 효율성을 높이는 방법에 대한 노하우를 담고 있습니다. 또한, LLM(대형 언어 모델) 서비스를 쉽게 만들고 이해할 수 있는 개발 과정도 배울 수 있는 콘텐츠를 제공합니다. 이 강의는 짧고 강력하며 핵심만을 전달하는 방식으로 진행됩니다. 수강생들은 강의가 재미있고 역동적이라고 하며, 많은 기술적 내용이 잘 설명되어 있어 추천한다고 평가하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 173, 'prompt_tokens': 2563, 'total_tokens': 2736, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CTEC40QrAQaIX6uEpNxEnjuNU8BhJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3971a7fc-2544-4ed0-8978-85ea606b5118-0', usage_m